In [1]:
from importlib import reload
import itertools
import os

import pandas as pd
import numpy as np

import main

In [26]:
reload(main)

<module 'main' from '/mnt/d/Study/ML/labs/lab1/main.py'>

In [6]:
os.chdir('lab1')

FileNotFoundError: [Errno 2] No such file or directory: 'lab1'

In [2]:
df = pd.read_csv('ex1data2.csv', header=None, names=['area', 'rooms', 'price'])
df = main.normalize(df)

In [3]:
theta_0 = pd.Series(0, index=range(len(df.columns)))
x = main.create_design_matrix(df[['area', 'rooms']])
y = df.price

In [4]:
def run(alpha, sensitivity, theta_0, x, y):
    prev_loss = None
    theta = None
    _progress = []
    for current_theta, loss in main.gradient_descent(theta_0, x, y, alpha):
        _progress.append((current_theta[0], current_theta[1], loss))
        delta = (prev_loss - loss) / loss if prev_loss is not None else sensitivity
        if delta < 0:
            raise ValueError('Loss increases. Decrease the learning rate')
        if delta < sensitivity:
            theta = current_theta
            break
        prev_loss = loss
    progress = pd.DataFrame.from_records(_progress, columns=['theta0', 'theta1', 'loss'])
    return progress


def run_safe(*args):
    try:
        return run(*args)
    except ValueError:
        return None

In [5]:
alpha = 1
sensitivity = 0.001
    
progress = run(alpha, sensitivity, theta_0, x, y)
print('Finished in {} iterations. Final loss: {}'.format(len(progress), progress.iloc[-1][2]))

Finished in 82 iterations. Final loss: 0.00747450025437374


In [6]:
def get_results():
    for alpha in np.logspace(-5, 1.5, 11):
        progress = run_safe(alpha, sensitivity, theta_0, x, y)
        if progress is not None:
            yield (alpha, len(progress), progress.loss.iloc[-1])

            
results = pd.DataFrame(get_results())

In [19]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

fig = plt.figure()
ax = fig.add_subplot(211, projection='3d')
ax.plot(
    results[0],
    results[1],
    results[2]
)

plt.xlabel('Learning rate')
plt.ylabel('Iteration count')

bx = fig.add_subplot(223)
bx.semilogx(results[0], results[2])

plt.xlabel('Learning rate')
plt.ylabel('Loss')

cx = fig.add_subplot(224)
cx.semilogx(results[0], results[1])

plt.xlabel('Learning rate')
plt.ylabel('Iteration count')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
analytic = main.analytic_solution(x, y)

loss = main.linear_loss_function(analytic, x, y)
print('Absolute minimum loss:', loss)

Absolute minimum loss: 0.0072740478839545325
